# Employee Attrition Prediction
## Notebook 02: Data Cleaning & Feature Engineering

This notebook covers:
- Loading raw datasets from Amazon S3
- Data cleaning and preprocessing
- Feature encoding and transformation
- Train/validation/test split
- Saving processed datasets back to S3

No model training is performed in this notebook.


In [1]:
import pandas as pd
import numpy as np

import sagemaker
from sagemaker.session import Session

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

import warnings
warnings.filterwarnings("ignore")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

print("Using bucket:", bucket)

Using bucket: sagemaker-us-east-1-952878272094


* Loading Raw Data from S3

In [3]:
train_s3_path = f"s3://{bucket}/employee-attrition/raw-data/train.csv"
test_s3_path  = f"s3://{bucket}/employee-attrition/raw-data/test.csv"

train_df = pd.read_csv(train_s3_path)
test_df  = pd.read_csv(test_s3_path)

print(train_df.shape, test_df.shape)

(59598, 24) (14900, 24)


* Dropping Non-informative columns

In [4]:
train_df.drop(columns=["Employee ID"], inplace=True)
test_df.drop(columns=["Employee ID"], inplace=True)

### Handling Missing Values

In [5]:
# Numerical columns
num_cols = train_df.select_dtypes(include=["int64", "float64"]).columns

for col in num_cols:
    train_df[col].fillna(train_df[col].median(), inplace=True)
    test_df[col].fillna(train_df[col].median(), inplace=True)

# Categorical columns
cat_cols = train_df.select_dtypes(include=["object"]).columns

for col in cat_cols:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)
    test_df[col].fillna(train_df[col].mode()[0], inplace=True)

### Ordinal Encoding (Manual & Explicit)

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59598 entries, 0 to 59597
Data columns (total 23 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       59598 non-null  int64 
 1   Gender                    59598 non-null  object
 2   Years at Company          59598 non-null  int64 
 3   Job Role                  59598 non-null  object
 4   Monthly Income            59598 non-null  int64 
 5   Work-Life Balance         59598 non-null  object
 6   Job Satisfaction          59598 non-null  object
 7   Performance Rating        59598 non-null  object
 8   Number of Promotions      59598 non-null  int64 
 9   Overtime                  59598 non-null  object
 10  Distance from Home        59598 non-null  int64 
 11  Education Level           59598 non-null  object
 12  Marital Status            59598 non-null  object
 13  Number of Dependents      59598 non-null  int64 
 14  Job Level             

In [7]:
ordinal_mappings = {
    "Work-Life Balance": {"Poor": 1, "Below Average": 2, "Good": 3, "Excellent": 4},
    "Job Satisfaction": {"Very Low": 1, "Low": 2, "Medium": 3, "High": 4},
    "Performance Rating": {"Low": 1, "Below Average": 2, "Average": 3, "High": 4},
    "Company Reputation": {"Very Poor": 1, "Poor": 2, "Good": 3, "Excellent": 4},
    "Employee Recognition": {"Very Low": 1, "Low": 2, "Medium": 3, "High": 4},
    "Job Level": {"Entry": 1, "Mid": 2, "Senior": 3},
    "Company Size": {"Small": 1, "Medium": 2, "Large": 3}
}

for col, mapping in ordinal_mappings.items():
    train_df[col] = train_df[col].map(mapping)
    test_df[col] = test_df[col].map(mapping)

In [8]:
pd.set_option('display.max_columns', None)

In [9]:
train_df.head(10)

,Age,Gender,Years at Company,Job Role,Monthly Income,Work-Life Balance,Job Satisfaction,Performance Rating,Number of Promotions,Overtime,Distance from Home,Education Level,Marital Status,Number of Dependents,Job Level,Company Size,Company Tenure,Remote Work,Leadership Opportunities,Innovation Opportunities,Company Reputation,Employee Recognition,Attrition
0,31,Male,19,Education,5390,4.0,3.0,3,2,No,22,Associate Degree,Married,0,2,2,89,No,No,No,4.0,3.0,Stayed
1,59,Female,4,Media,5534,1.0,4.0,1,3,No,21,Master’s Degree,Divorced,3,2,2,21,No,No,No,NaN,2.0,Stayed
2,24,Female,10,Healthcare,8159,3.0,4.0,1,0,No,11,Bachelor’s Degree,Married,3,2,2,74,No,No,No,2.0,2.0,Stayed
3,36,Female,7,Education,3989,3.0,4.0,4,1,No,27,High School,Single,2,2,1,50,Yes,No,No,3.0,3.0,Stayed
4,56,Male,41,Education,4821,NaN,NaN,3,0,Yes,71,High School,Divorced,0,3,2,68,No,No,No,NaN,3.0,Stayed
5,38,Female,3,Technology,9977,NaN,4.0,2,3,No,37,Bachelor’s Degree,Married,0,2,2,47,No,No,Yes,NaN,4.0,Left
6,47,Male,23,Education,3681,NaN,4.0,4,1,Yes,75,High School,Divorced,3,1,1,93,No,No,No,3.0,3.0,Left
7,48,Male,16,Finance,11223,4.0,NaN,4,2,No,5,Master’s Degree,Married,4,1,2,88,No,No,No,4.0,2.0,Stayed
8,57,Male,44,Education,3773,3.0,3.0,4,1,Yes,39,High School,Married,4,1,2,75,No,No,No,NaN,3.0,Stayed
9,24,Female,1,Healthcare,7319,1.0,4.0,3,1,Yes,57,PhD,Single,4,1,3,45,No,No,Yes,3.0,2.0,Left


In [10]:
# Handling NaNs introduced by ordinal mapping
ordinal_cols = list(ordinal_mappings.keys())

for col in ordinal_cols:
    train_df[col].fillna(train_df[col].median(), inplace=True)
    test_df[col].fillna(train_df[col].median(), inplace=True)

### Binary Encoding (Yes / No)

In [11]:
def identify_binary_categorical_columns(df):
    binary_cols = []
    for col in df.select_dtypes(include=["object"]).columns:
        unique_vals = df[col].dropna().unique()
        if len(unique_vals) == 2:
            binary_cols.append(col)
    return binary_cols

identify_binary_categorical_columns(train_df)

['Gender',
 'Overtime',
 'Remote Work',
 'Leadership Opportunities',
 'Innovation Opportunities',
 'Attrition']

In [12]:
target_col = "Attrition"

# the categories for the below column can increase in the future
nominal_binary_cols = ["Gender"]

In [13]:
binary_cols = [
    col for col in identify_binary_categorical_columns(train_df)
    if col != target_col and col not in nominal_binary_cols
]

for col in binary_cols:
    unique_vals = train_df[col].dropna().unique()
    
    mapping = {
        unique_vals[0]: 0,
        unique_vals[1]: 1
    }
    
    train_df[col] = train_df[col].map(mapping)
    test_df[col]  = test_df[col].map(mapping)

### One-Hot Encoding (Nominal Categories)

In [14]:
nominal_cols = [
    "Gender",
    "Job Role",
    "Education Level",
    "Marital Status"
]

train_df = pd.get_dummies(train_df, columns=nominal_cols, drop_first=True)
test_df  = pd.get_dummies(test_df, columns=nominal_cols, drop_first=True)

# Aligning train & test columns
train_df, test_df = train_df.align(test_df, join="left", axis=1, fill_value=0)

### Splitting Features & Target

In [15]:
# Encode target variable explicitly
train_df["Attrition"] = train_df["Attrition"].map({
    "Stayed": 0,
    "Left": 1
})

In [16]:
X = train_df.drop(columns=["Attrition"])
y = train_df["Attrition"]

### Train / Validation / Test Split

In [17]:
X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.30, stratify=y, random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.50, stratify=y_temp, random_state=42
)

print(X_train.shape, X_val.shape, X_test.shape)

(41718, 29) (8940, 29) (8940, 29)


In [18]:
# Combining labels and features for SageMaker XGBoost
train_combined = pd.concat([y_train, X_train], axis=1)
val_combined   = pd.concat([y_val, X_val], axis=1)

In [19]:
y_train.unique(), y_val.unique()

(array([0, 1]), array([1, 0]))

In [31]:
train_combined.head()

,Attrition,Age,Years at Company,Monthly Income,Work-Life Balance,Job Satisfaction,Performance Rating,Number of Promotions,Overtime,Distance from Home,Number of Dependents,Job Level,Company Size,Company Tenure,Remote Work,Leadership Opportunities,Innovation Opportunities,Company Reputation,Employee Recognition,Gender_Male,Job Role_Finance,Job Role_Healthcare,Job Role_Media,Job Role_Technology,Education Level_Bachelor’s Degree,Education Level_High School,Education Level_Master’s Degree,Education Level_PhD,Marital Status_Married,Marital Status_Single
37648,0,56,42,5513,4.0,4.0,3,0,1,62,5,1,2,106,0,0,0,4.0,4.0,False,False,False,True,False,False,False,True,False,True,False
13474,1,37,25,5135,3.0,4.0,1,0,0,75,2,1,3,74,0,0,1,3.0,2.0,True,False,False,False,False,True,False,False,False,True,False
31893,0,31,15,11430,1.0,3.0,3,1,0,9,1,2,2,85,1,0,0,3.0,3.0,False,False,False,False,True,False,False,False,False,True,False
14595,1,42,31,8979,3.0,4.0,3,2,1,30,0,1,3,96,0,0,0,3.0,3.0,True,False,False,False,True,False,False,False,True,False,False
15116,0,31,20,9003,1.0,4.0,3,0,1,88,1,1,1,78,1,0,1,3.0,4.0,True,False,False,False,True,False,False,True,False,False,False


### Feature Scaling (Numeric Only)

In [20]:
numeric_cols_to_scale = [
    "Age",
    "Years at Company",
    "Monthly Income",
    "Number of Promotions",
    "Distance from Home",
    "Number of Dependents",
    "Company Tenure"
]

In [21]:
scaler = StandardScaler()

X_train[numeric_cols_to_scale] = scaler.fit_transform(X_train[numeric_cols_to_scale])
X_val[numeric_cols_to_scale]   = scaler.transform(X_val[numeric_cols_to_scale])
X_test[numeric_cols_to_scale]  = scaler.transform(X_test[numeric_cols_to_scale])

In [22]:
import joblib

scaler_path = "/tmp/standard_scaler.joblib"
joblib.dump(scaler, scaler_path)

sagemaker_session.upload_data(
    path=scaler_path,
    bucket=bucket,
    key_prefix="employee-attrition/artifacts"
)

's3://sagemaker-us-east-1-952878272094/employee-attrition/artifacts/standard_scaler.joblib'

### Saving Processed Data to S3

In [23]:
processed_prefix = "employee-attrition/processed"

train_path = f"s3://{bucket}/{processed_prefix}/train.csv"
val_path   = f"s3://{bucket}/{processed_prefix}/validation.csv"
test_path  = f"s3://{bucket}/{processed_prefix}/test.csv"

train_combined.to_csv(train_path, index=False, header=False)
val_combined.to_csv(val_path, index=False, header=False)
X_test.to_csv(test_path, index=False, header=False)

print("SageMaker-compatible datasets saved to S3")

SageMaker-compatible datasets saved to S3


In [24]:
# Shape sanity check (most important)
print("X_train:", X_train.shape)
print("X_val  :", X_val.shape)
print("X_test :", X_test.shape)

print("y_train:", y_train.shape)
print("y_val  :", y_val.shape)
print("y_test :", y_test.shape)

X_train: (41718, 29)
X_val  : (8940, 29)
X_test : (8940, 29)
y_train: (41718,)
y_val  : (8940,)
y_test : (8940,)


In [25]:
# NaN check
print("NaNs in X_train:", X_train.isnull().sum().sum())
print("NaNs in X_val  :", X_val.isnull().sum().sum())
print("NaNs in X_test :", X_test.isnull().sum().sum())

NaNs in X_train: 0
NaNs in X_val  : 0
NaNs in X_test : 0


In [26]:
# Target balance check (stratification validation)
print("Train target distribution:")
print(y_train.value_counts(normalize=True))

print("\nValidation target distribution:")
print(y_val.value_counts(normalize=True))

print("\nTest target distribution:")
print(y_test.value_counts(normalize=True))

Train target distribution:
Attrition
0    0.524522
1    0.475478
Name: proportion, dtype: float64

Validation target distribution:
Attrition
0    0.524497
1    0.475503
Name: proportion, dtype: float64

Test target distribution:
Attrition
0    0.524497
1    0.475503
Name: proportion, dtype: float64


In [27]:
# Feature count consistency
print("Feature count:")
print(X_train.shape[1], X_val.shape[1], X_test.shape[1])

Feature count:
29 29 29


In [28]:
for df in [X_train, X_val, X_test]:
    bool_cols = df.select_dtypes(include=["bool"]).columns
    df[bool_cols] = df[bool_cols].astype(int)

### Summary

- Raw data loaded from Amazon S3
- Missing values handled
- Ordinal, binary, and nominal features encoded
- Data split into train, validation, and test sets
- Numerical features scaled
- Processed datasets saved back to Amazon S3